In [1]:
from backtest_lib.backtest import Backtest
import datetime as dt
from backtest_lib.examples.get_sp500_historical import get_sp500_market_view
import numpy as np
from backtest_lib.market.polars_impl import SeriesUniverseMapping
from backtest_lib.market.polars_impl import PolarsPastView
from backtest_lib.market import MarketView, PastUniversePrices
import polars as pl
import pickle as pkl
from importlib.resources import files
import backtest_lib.examples
from backtest_lib.portfolio import WeightedPortfolio
from backtest_lib.strategy import Decision
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,  # Jupyter displays stdout cleanly
    format="[%(levelname)s] %(name)s: %(message)s",
    force=True,  # overwrite any existing handlers
)

market_view = get_sp500_market_view(
    start=dt.datetime.fromisoformat("2022-12-23"),
    end=dt.datetime.fromisoformat("2025-10-21"),
)

universe = tuple(market_view.prices.close.by_security.as_df().columns)


In [10]:
pf = WeightedPortfolio(
    cash=0,
    holdings=SeriesUniverseMapping.from_names_and_data(
        universe, pl.Series([1 / len(universe)] * len(universe))
    ),
)


def get_even_portfolio(full_universe, partial_universe: list[str]) -> WeightedPortfolio:
    zeroed_securities = list(set(full_universe) - set(partial_universe))
    return WeightedPortfolio(
        cash=0,
        holdings=SeriesUniverseMapping.from_names_and_data(
            partial_universe + zeroed_securities,
            pl.Series(
                ([1 / len(partial_universe)] * len(partial_universe))
                + ([0.0] * len(zeroed_securities))
            ),
        ),
    )


def index_strategy(universe, current_portfolio, market, ctx) -> Decision:
    tradable = [x for x, y in market.tradable.by_period[-1].items() if y]
    even_portfolio = get_even_portfolio(universe, tradable)

    return Decision(even_portfolio)


df = market_view.prices.close.by_security.as_df()
cols = df.columns
ewma = (
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in cols)
    .select(cols)
    .with_columns(date=market_view.periods)
)

ewma_past_view = PolarsPastView.from_data_frame(ewma)

market_view = MarketView(
    prices=market_view.prices,
    tradable=market_view.tradable,
    signals={"ewma_ratio": ewma_past_view},
    periods=market_view.periods,
)

def ewma_strategy(universe, current_portfolio, market, ctx):
    
    tradable_today = market.tradable.by_period[-1]
    latest_ewma_ratio = (
        market.signals["ewma_ratio"]
        .by_security[[sec for sec in universe if tradable_today[sec] is True]]
        .by_period[-1]
    )
    
    avg_ewma_ratio = latest_ewma_ratio.mean()
    norm_ewma_ratio = latest_ewma_ratio / avg_ewma_ratio
    weights = norm_ewma_ratio / len(norm_ewma_ratio)
    return Decision(target=WeightedPortfolio(cash=0, holdings=weights))


benchmark_backtest = Backtest(
    universe=universe,
    strategy=index_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

ewma_backtest = Backtest(
    universe=universe,
    strategy=ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

In [ ]:
ewma_results = ewma_backtest.run()
print(f"total return: {(ewma_results.total_growth - 1) * 100:.2f}%")

In [ ]:
results = benchmark_backtest.run()
print(f"total return: {(results.total_growth - 1) * 100:.2f}%")

In [ ]:
market_view.tradable.by_period